## KIPRIS Plus OpenAPI Quickstart (키 기반)

이 노트북은 **KIPRIS Plus OpenAPI**로 특허 데이터를 검색/수집하는 절차를 확인하기 위한 예제입니다.

- 참고(공식): [KIPRIS Plus API 통합설명서](https://plus.kipris.or.kr/portal/bbs/view.do?bbsId=B0000004&delCode=&edate=&menuNo=&nttId=858&pageIndex=1&replyAt=&sdate=&searchCnd=&searchWrd=&section=&useAt=&viewType=)
- 참고(오픈소스): [mcp_kipris](https://github.com/nuri428/mcp_kipris) - KIPRIS API 래퍼

### 준비
- 프로젝트 루트에 `env` 또는 `.env` 파일을 만들고 아래 값을 채우세요.
  - `KIPRIS_API_KEY=...`
  - (선택) `KIPRIS_BASE_URL=http://plus.kipris.or.kr/kipo-api`
  - (선택) `KIPRIS_PATENT_SEARCH_PATH=kipi/patUtiModInfoSearchSevice/getAdvancedSearch`

> **주의**: API 도메인이 **HTTP**(HTTPS 아님)입니다. 키를 노트북/레포에 하드코딩하지 마세요.


In [1]:
import os
import time
from typing import Any, Dict, List, Optional, Tuple

import requests
import pandas as pd
from dotenv import load_dotenv

# 키 로드(명시 로드): 환경/실행방식에 따라 자동 탐색이 불안정할 수 있어 명시합니다.
load_dotenv(".env")
load_dotenv("env")

KIPRIS_API_KEY = os.getenv("KIPRIS_API_KEY", "")
# KIPRIS API 기본 도메인 (HTTP 사용, HTTPS 아님!)
KIPRIS_BASE_URL = os.getenv("KIPRIS_BASE_URL", "http://plus.kipris.or.kr/kipo-api")

# KIPRIS 특허/실용신안 검색 엔드포인트 (mcp_kipris 프로젝트 기준 확인됨)
KIPRIS_PATENT_SEARCH_PATH = os.getenv(
    "KIPRIS_PATENT_SEARCH_PATH",
    "kipi/patUtiModInfoSearchSevice/getAdvancedSearch",
)

if not KIPRIS_API_KEY:
    raise ValueError("KIPRIS_API_KEY가 비어 있습니다. 프로젝트 루트의 env/.env에 설정하세요.")

SESSION = requests.Session()
SESSION.headers.update({"Accept": "*/*", "User-Agent": "kipris-notebook/0.1"})

print("KIPRIS_BASE_URL:", KIPRIS_BASE_URL)
print("KIPRIS_PATENT_SEARCH_PATH:", KIPRIS_PATENT_SEARCH_PATH)
print("API KEY loaded:", KIPRIS_API_KEY[:4] + "..." + KIPRIS_API_KEY[-4:])


KIPRIS_BASE_URL: http://plus.kipris.or.kr/kipo-api
KIPRIS_PATENT_SEARCH_PATH: kipi/patUtiModInfoSearchSevice/getAdvancedSearch
API KEY loaded: aggT...Xas=


In [2]:
# 엔드포인트 진단 셀
# - 지금처럼 HTML(404/오류 페이지)이 오면, 'path가 틀렸거나(변경/오타) / 접근이 차단'된 상태입니다.
# - KIPRIS에서 제공한 "호출 URL" 예시가 있으면 여기에 넣고 결과(content-type)가 XML/JSON으로 나오는지 확인하세요.

import re


def _extract_html_title(text: str) -> str:
    m = re.search(r"<title>(.*?)</title>", text, flags=re.IGNORECASE | re.DOTALL)
    if not m:
        return ""
    return re.sub(r"\s+", " ", m.group(1)).strip()[:120]


def probe(path: str) -> None:
    url = f"{KIPRIS_BASE_URL.rstrip('/')}/{path.lstrip('/')}"
    # 새로운 파라미터 스키마 (word 기반)
    params = {
        "ServiceKey": KIPRIS_API_KEY,
        "word": "LLM",
        "patent": "true",
        "utility": "true",
        "numOfRows": "1",
        "pageNo": "1",
    }
    r = SESSION.get(url, params=params, timeout=30, allow_redirects=True)
    ct = r.headers.get("content-type", "")
    head = (r.text or "").lstrip()[:200]
    title = _extract_html_title(r.text or "")
    print("- url:", url)
    print("  status:", r.status_code, "final:", r.url)
    print("  content-type:", ct)
    if "text/html" in ct.lower() or head.lower().startswith("<html") or head.lower().startswith("<!doctype html"):
        print("  html title:", title)
    else:
        # XML 응답이면 successYN, resultCode 추출
        success = re.search(r"<successYN>([^<]+)", head)
        code = re.search(r"<resultCode>([^<]+)", head)
        msg = re.search(r"<resultMsg>([^<]+)", head)
        print("  successYN:", success.group(1) if success else "?")
        print("  resultCode:", code.group(1).strip() if code else "?")
        print("  resultMsg:", msg.group(1) if msg else "?")


# 현재 설정값으로 1회 진단
probe(KIPRIS_PATENT_SEARCH_PATH)

# 필요하면 후보를 추가해서 비교해보세요(문서에 나온 정확한 path가 있으면 그걸 넣는게 정답입니다)
# for p in [
#     "openapi/rest/.../searchPatentInfo",
# ]:
#     probe(p)



- url: http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch
  status: 200 final: http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice/getAdvancedSearch?ServiceKey=aggTFeK8ph%3DmkUCwTp%3DNNMl77qysvUvLk6ruJi%3DjXas%3D&word=LLM&patent=true&utility=true&numOfRows=1&pageNo=1
  content-type: text/xml;charset=utf-8
  successYN: Y
  resultCode: ?
  resultMsg: ?


In [3]:
import json
import xml.etree.ElementTree as ET


def _try_parse_json(text: str) -> Optional[Dict[str, Any]]:
    try:
        return json.loads(text)
    except Exception:
        return None


def _looks_like_html(raw: str, content_type: str) -> bool:
    if "text/html" in (content_type or "").lower():
        return True
    head = (raw or "").lstrip()[:200].lower()
    return head.startswith("<!doctype html") or head.startswith("<html")


def _xml_items_to_rows(xml_text: str) -> List[Dict[str, Any]]:
    """KIPRIS OpenAPI가 XML로 응답하는 경우를 대비한 최소 파서.

    - 응답 구조는 서비스별로 다를 수 있어, 여기서는 <item> 하위의 1-depth 텍스트 노드만 평탄화합니다.
    - XML이 아닌 HTML(오류 페이지) 등을 받으면 ParseError가 날 수 있어, 호출부에서 먼저 HTML을 차단합니다.
    """
    root = ET.fromstring(xml_text)
    rows: List[Dict[str, Any]] = []
    for item in root.findall(".//item"):
        row: Dict[str, Any] = {}
        for child in list(item):
            tag = child.tag
            val = (child.text or "").strip()
            row[tag] = val
        if row:
            rows.append(row)
    return rows


def kipris_get(path: str, params: Dict[str, Any], *, timeout: int = 60) -> Tuple[List[Dict[str, Any]], Dict[str, Any], str]:
    """KIPRIS OpenAPI 호출(GET).

    반환:
    - rows: item list를 평탄화한 레코드
    - meta: count/페이지 등 메타(있으면)
    - raw_text: 원문(디버깅/재현용)

    중요:
    - KIPRIS가 XML/JSON 대신 HTML(404/오류 페이지)을 반환하면, XML 파싱 전에 친절한 에러로 중단합니다.
    """
    url = f"{KIPRIS_BASE_URL.rstrip('/')}/{path.lstrip('/')}"
    r = SESSION.get(url, params=params, timeout=timeout)
    raw = r.text
    content_type = r.headers.get("content-type", "")

    if r.status_code >= 400:
        raise RuntimeError(f"HTTP {r.status_code} ({content_type}) url={url} body={raw[:300]}")

    # KIPRIS가 '페이지를 찾을 수 없습니다' 같은 HTML을 200으로 주는 경우가 있어 선제 차단
    if _looks_like_html(raw, content_type):
        # 타이틀/핵심 문구만 뽑아 힌트를 줌
        head = " ".join(raw.splitlines()[:40])
        raise RuntimeError(
            "KIPRIS OpenAPI 응답이 XML/JSON이 아니라 HTML(오류 페이지)입니다. "
            "즉, 현재 KIPRIS_BASE_URL 또는 path가 실제 OpenAPI 엔드포인트가 아니거나(변경/오타), "
            "네트워크/권한(기관측 설정) 문제로 OpenAPI가 차단된 상태입니다. "
            f"url={url} content-type={content_type} head={head[:300]}"
        )

    # JSON이면 우선 JSON으로
    obj = _try_parse_json(raw)
    if obj is not None:
        items = (
            obj.get("response", {}).get("body", {}).get("items")
            or obj.get("body", {}).get("items")
            or obj.get("items")
            or []
        )
        if isinstance(items, dict) and "item" in items:
            items = items["item"]
        if isinstance(items, dict):
            items = [items]
        meta = {"parsed": "json", "raw_keys": list(obj.keys())}
        return list(items) if isinstance(items, list) else [], meta, raw

    # XML이면 item 파싱
    try:
        rows = _xml_items_to_rows(raw)
    except ET.ParseError as e:
        raise RuntimeError(
            "XML 파싱 실패(응답이 XML 스펙과 다르거나 잘린 경우). "
            f"url={url} content-type={content_type} err={e} body_head={raw[:300]}"
        )

    meta = {"parsed": "xml", "count": len(rows)}
    return rows, meta, raw


def kipris_keyword_search_patent(
    *,
    word: str,
    patent: bool = True,
    utility: bool = True,
    num_of_rows: int = 20,
    page_no: int = 1,
    desc_sort: bool = False,
    sort_spec: str = "AD",
    sleep_sec: float = 0.2,
    path: str = "kipi/patUtiModInfoSearchSevice/getAdvancedSearch",
    **extra_params,
) -> List[Dict[str, Any]]:
    """KIPRIS 특허/실용신안 고급 검색.

    Args:
        word: 자유검색 키워드
        patent: 특허 포함 여부
        utility: 실용신안 포함 여부
        num_of_rows: 페이지당 결과 수
        page_no: 페이지 번호
        desc_sort: 내림차순 정렬
        sort_spec: 정렬 기준 (AD=출원일 등)
        sleep_sec: 호출 후 대기 시간
        path: API 엔드포인트 경로
        **extra_params: 추가 검색 조건 (inventionTitle, abstCont 등)

    Returns:
        특허 레코드 목록
    """
    params = {
        "ServiceKey": KIPRIS_API_KEY,
        "word": word,
        "patent": "true" if patent else "false",
        "utility": "true" if utility else "false",
        "numOfRows": str(num_of_rows),
        "pageNo": str(page_no),
        "descSort": "true" if desc_sort else "false",
        "sortSpec": sort_spec,
    }
    # 추가 파라미터 (camelCase로 전달)
    for k, v in extra_params.items():
        if v is not None and v != "":
            # snake_case -> camelCase 변환
            parts = k.split("_")
            camel = parts[0] + "".join(p.capitalize() for p in parts[1:])
            params[camel] = str(v)

    rows, _meta, _raw = kipris_get(path, params)
    time.sleep(sleep_sec)
    return rows


In [4]:
# 샘플 1) LLM / Agentic AI 관련 키워드
keywords_ai = [
    "대규모 언어 모델",
    "LLM",
    "에이전트",
    "자율 에이전트",
    "검색 증강 생성",
    "RAG",
    "인공지능",
]

rows_ai = []
for kw in keywords_ai:
    try:
        rows = kipris_keyword_search_patent(
            word=kw,
            num_of_rows=10,
            page_no=1,
            path=KIPRIS_PATENT_SEARCH_PATH,
        )
        print(kw, "rows=", len(rows))
        rows_ai.extend(rows)
    except Exception as e:
        print("FAIL", kw, e)

# 중복 제거(출원번호 기준)
df_ai = pd.DataFrame(rows_ai)
if not df_ai.empty and "applicationNumber" in df_ai.columns:
    df_ai = df_ai.drop_duplicates(subset=["applicationNumber"])
display(df_ai.head(10))
print("columns:", list(df_ai.columns)[:40])


대규모 언어 모델 rows= 10
LLM rows= 10
에이전트 rows= 10
자율 에이전트 rows= 10
검색 증강 생성 rows= 10
RAG rows= 10
인공지능 rows= 10


,applicantName,applicationDate,applicationNumber,astrtCont,bigDrawing,drawing,indexNo,inventionTitle,ipcNumber,openDate,openNumber,publicationDate,publicationNumber,registerDate,registerNumber,registerStatus
0,알카테엔브이,19840308,1019840001179,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,1,연상어레이,G06F 15/00,19841213,1019840008190,19911028,1019910009095,19920427,1000510760000,소멸
1,로리에이.요한슨,19861016,1019880700680,내용 없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,2,자외선방사및청색광차단용편광렌즈,G02C 7/10,19881105,1019880701894,19940217,1019940001224,19940810,1000762010000,소멸
2,훠뮤랩인터내셔날리미티드,19880129,1019890700439,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,3,인지시스템용아키텍처,G06F 12/00|G16C 10/00,19891223,1019890702150,19980615,,19980130,1001368770000,소멸
3,파나소닉 전공 주식회사,19880712,1019880008653,내용없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,4,음성인코딩및합성시스템,G10L 13/02|G06K 7/10|G09B 5/06|G09B 21/00,19890411,1019890002815,19910520,1019910003144,19910828,1000438880000,소멸
4,3디 시스템즈 인코오퍼레이티드,19890123,1019970709891,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,5,CAD/CAM스테레오리소그래픽데이타변환,B29C 64/135|G06T 17/10|B29C 64/40|B44B 1/00|G0...,19900813,1019900700275,19991115,,19990817,1002295800000,소멸
5,3디 시스템즈 인코오퍼레이티드,19890417,1019890702387,"본 발명은, 적절한 상승적인 자극, 커얼을 감소시키기 위하여 특별히 처리되고 있는 ...",http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,6,스테레오리소그래피를 사용하여 3차원 물체를 형성하기 위한 방법 및 장치,B29C 35/08,19900811,1019900700191,20000601,,20000226,1002570330000,소멸
6,3디 시스템즈 인코오퍼레이티드,19890417,1019970709511,"본 발명은, 충돌 조사, 입자 충격 또는 화학 반응에 의한 적절한 상승작용적 자극에...",http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,7,스테레오리소그래픽커얼감소,B29C 35/08|B29C 67/00,19990101,1019970520000,19990515,,19981125,1001788730000,소멸
7,3디 시스템즈 인코오퍼레이티드,19890417,1019997002910,요약 누락,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,8,캐드/캠 스테레오리소그래픽 데이타 변환,B29C 37/00,,,20010115,,20001114,1002809860000,소멸
8,3디 시스템즈 인코오퍼레이티드,19890417,1019890702388,“스테레오리소그래픽”은 예를들면 자외선 경화성물질과 같이 경화성물질의 얇은 층을 연...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,9,데이타 처리를 포함하는 스테레오리소그래피를 이용한 3차원 물체 형성방법 및 장치,B29C 67/24,19900813,1019900700275,20000601,,20000226,1002570340000,소멸
9,똥송-쎄 에스 에프,19891020,1019900701315,내용없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,10,"변조파 전송방법 및 이 방법을 구현하기 위한 송,수신기",H04L 5/02,19901208,1019900702684,19980701,,19980303,1001395270000,소멸


columns: ['applicantName', 'applicationDate', 'applicationNumber', 'astrtCont', 'bigDrawing', 'drawing', 'indexNo', 'inventionTitle', 'ipcNumber', 'openDate', 'openNumber', 'publicationDate', 'publicationNumber', 'registerDate', 'registerNumber', 'registerStatus']


In [5]:
# 샘플 2) 반도체 장비 관련 키워드
keywords_semi = [
    "반도체 장비",
    "노광",
    "리소그래피",
    "식각",
    "플라즈마 식각",
    "CVD",
    "PVD",
    "원자층 증착",
    "ALD",
]

rows_semi = []
for kw in keywords_semi:
    try:
        rows = kipris_keyword_search_patent(
            word=kw,
            num_of_rows=10,
            page_no=1,
            path=KIPRIS_PATENT_SEARCH_PATH,
        )
        print(kw, "rows=", len(rows))
        rows_semi.extend(rows)
    except Exception as e:
        print("FAIL", kw, e)

df_semi = pd.DataFrame(rows_semi)
if not df_semi.empty and "applicationNumber" in df_semi.columns:
    df_semi = df_semi.drop_duplicates(subset=["applicationNumber"])
display(df_semi.head(10))
print("columns:", list(df_semi.columns)[:40])


반도체 장비 rows= 10
노광 rows= 10
리소그래피 rows= 10
식각 rows= 10
플라즈마 식각 rows= 10
CVD rows= 10
PVD rows= 10
원자층 증착 rows= 10
ALD rows= 10


,applicantName,applicationDate,applicationNumber,astrtCont,bigDrawing,drawing,indexNo,inventionTitle,ipcNumber,openDate,openNumber,publicationDate,publicationNumber,registerDate,registerNumber,registerStatus
0,텍사스인스터러먼츠인코포레이티드,19760412,1019760000881,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,1,반도체장치의제조방법,H01L 21/266|H10D 1/20,,,19801026,1019800001272,19810116,1000091880000,소멸
1,인터내쇼날 스탠다드 일렉트릭 코포레이션,19780328,1019780000862,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,2,스위칭시스템위한분배제어,H04Q 3/68|H04Q 11/04,,,19830518,1019830000982,19830801,1000149190000,소멸
2,텍사스인스터러먼츠인코포레이티드,19780529,1019780001617,내용 없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,3,디지탈필터,H03H 17/02|H03H 17/04|G10L 13/047,19830428,1019830000955,19841221,1019840002361,19850327,1000191300000,소멸
3,씨비에스 코퍼레이션,19780817,1019780002516,내용 없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,4,견인모터전류제어장치,B60L 15/04|B60L 15/20|H02P 7/29,19830330,1019830000466,19840409,1019840000479,19840625,1000171700000,소멸
4,아세아악티에볼락,19780922,1019780002898,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,5,전기차량견인장치,B60L 9/04|B60L 3/00,,,19820728,1019820001350,19821112,1000131430000,소멸
5,도오요오 세이깡 가부시끼가이샤,19790525,1019790001695,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,6,금속캡의연속가열장치,C21D 9/00|C21D 1/42,,,19821012,1019820001822,19830930,1000153130000,소멸
6,에어픽스푸로덕트스리미티드,19790725,1019790002527,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,7,전동장치제어방법,B60L 15/00,,,19820921,1019820001710,19821227,1000134390000,소멸
7,더 링컨 일렉트릭 컴패니,19790828,1019790002959,내용 없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,8,자동프레셋트용접장치,B23K 9/12|B23K 9/10,19830429,1019830001014,19841214,1019840002263,19850320,1000190640000,소멸
8,두산투울즈인코오포레이티나,19790914,1019790003172,내용 없음.,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,9,분리식공동판으로구성된캡슐화주형,H01L 23/31|H01L 23/29|H01L 21/56,,,19830406,1019830000729,19830622,1000146220000,소멸
9,씨비에스 코퍼레이션,19800503,1019800001787,내용 없음,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,http://plus.kipris.or.kr/openapi/fileToss.jsp?...,10,원자로에있어서의물체충돌의검출방법,G21C 9/00,19830531,1019830003108,19831206,1019830002645,19840326,1000166540000,소멸


columns: ['applicantName', 'applicationDate', 'applicationNumber', 'astrtCont', 'bigDrawing', 'drawing', 'indexNo', 'inventionTitle', 'ipcNumber', 'openDate', 'openNumber', 'publicationDate', 'publicationNumber', 'registerDate', 'registerNumber', 'registerStatus']


In [6]:
from pathlib import Path

out_dir = Path("/home/arkwith/SKKU/paper_data/data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

# 저장: 컬럼이 서비스별로 달라질 수 있어 JSONL로 그대로 저장
(df_ai if not df_ai.empty else pd.DataFrame([])).to_json(
    out_dir / "kipris_ai_sample.jsonl", orient="records", lines=True, force_ascii=False
)
(df_semi if not df_semi.empty else pd.DataFrame([])).to_json(
    out_dir / "kipris_semiconductor_equipment_sample.jsonl", orient="records", lines=True, force_ascii=False
)

print("saved:", out_dir / "kipris_ai_sample.jsonl")
print("saved:", out_dir / "kipris_semiconductor_equipment_sample.jsonl")


saved: /home/arkwith/SKKU/paper_data/data/processed/kipris_ai_sample.jsonl
saved: /home/arkwith/SKKU/paper_data/data/processed/kipris_semiconductor_equipment_sample.jsonl


### 참고 및 트러블슈팅

- KIPRIS OpenAPI는 서비스별로 **엔드포인트/파라미터/응답 구조(XML/JSON)**가 다를 수 있습니다.
- 이 노트북은 우선 범용 확인용으로 작성되어, **JSON이면 JSON으로**, 아니면 **XML의 `<item>` 목록을 평탄화**해 DataFrame으로 만듭니다.
- 만약 `HTTP 400/403` 등이 나오면 아래를 점검하세요.
  - `KIPRIS_API_KEY`가 올바른지
  - 엔드포인트(path)가 사용 중인 서비스와 맞는지
  - 파라미터 이름이 문서와 일치하는지(`ServiceKey`, `searchKeyword`, `searchCondition` 등)

공식 가이드:
- [KIPRIS Plus API 통합설명서](https://plus.kipris.or.kr/portal/bbs/view.do?bbsId=B0000004&delCode=&edate=&menuNo=&nttId=858&pageIndex=1&replyAt=&sdate=&searchCnd=&searchWrd=&section=&useAt=&viewType=)
